<a href="https://colab.research.google.com/github/buistung/Advanced-Virtual-Assistant/blob/main/advanced_virtual_assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!CMAKE_ARGS="-DLLAMA_BLAS=ON -DLLAMA_BLAS_VENDOR=OpenBLAS" pip install llama-cpp-python

In [ ]:
!pip3 install huggingface-hub


In [ ]:
!huggingface-cli download TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF \
tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf \
--local-dir . \
--local-dir-use-symlinks False

In [ ]:
!pip install torch transformers

In [ ]:
!pip install bitsandbytes accelerate

In [ ]:
from llama_cpp import Llama
import torch
from transformers import pipeline
from transformers import AutoTokenizer


class Assistant():
    def __init__(self, model="TinyLlama/TinyLlama-1.1B-Chat-v1.0" ):
        self.pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.bfloat16, device_map="auto")
        self.tokenizer = AutoTokenizer.from_pretrained(model)
    def format_messages(self, user_input):
        messages = [
            {
                "role": "system",
                "content": "",
            },
            {"role": "user", "content": user_input},
        ]
        return self.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)


    def generate_response(self, user_input, max_new_tokens=128, temperature=0.5, top_k=1):
        prompt = self.format_messages(user_input)
        outputs = self.pipe(prompt, max_new_tokens=max_new_tokens, do_sample=True, temperature=temperature, top_k=top_k,)
        return outputs[0]["generated_text"]

    def extract_assistant_response(self, text):
        start_tag = "<|assistant|>"
        start_index = text.find(start_tag)
        if start_index == -1:
            return "No assistant response found"
        extracted_text = text[start_index + len(start_tag):].strip()
        return extracted_text

    def chat(self):
        print("I am a virtual assitant, type 'exit' to quit the program")
        print("---------------")
        print('Assistant: Hello, how can i help you?')
        print("---------------")
        while True:
            user_input = input('User: ')
            if user_input.lower() == 'exit':
                print('Good bye, have a great day!')
                break
            else:
                response = self.generate_response(user_input)
                extracted_text = self.extract_assistant_response(response)
                print("---------------")
                print(f"Assistant: {extracted_text}")
                print("---------------")

if __name__ == "__main__":
    assistant = Assistant()
    assistant.chat()